In [18]:
import numpy as np
from scipy.optimize import least_squares

# Constants
c = 299792458  # Speed of light in vacuum, in m/s

# Satellite positions in ECEF coordinates (x, y, z) for satellites 1 to 4
satellite_positions = np.array([
    [10053355.10, 15056487.96, 25504822.96],
    [20655550.99, 5147316.42, 16608524.31],
    [21764223.92, -6594586.14, 15678237.26],
    [10509172.26, -14025870.89, 25122346.95]
])

# Pseudo-range measurements for satellites 1 to 4
measured_pseudo_ranges = np.array([26155755.9003033, 21053427.7312908, 21833394.9590138, 25313055.4481566])

# Function to calculate the predicted pseudo-ranges
def calculate_predicted_ranges(position, clock_error, satellite_positions):
    ranges = np.sqrt(np.sum((satellite_positions - position) ** 2, axis=1))
    return ranges + clock_error

# Function to form the H matrix
def form_H_matrix(position, satellite_positions, predicted_ranges):
    H = np.zeros((len(satellite_positions), 4))
    for i, (sx, sy, sz) in enumerate(satellite_positions):
        r = predicted_ranges[i]
        H[i, :3] = (position - np.array([sx, sy, sz])) / r
        H[i, 3] = 1  # Derivative with respect to clock bias is 1
    return H

# Function to calculate the theoretical pseudo-ranges for a given receiver position and clock error
def calculate_pseudo_ranges(position, clock_error, satellite_positions):
    distances = np.sqrt(np.sum((satellite_positions - position) ** 2, axis=1))
    return distances + clock_error * c

# Residuals function for least squares solver
def residuals(params, satellite_positions, measured_pseudo_ranges):
    position = params[:3]
    clock_error = params[3]
    predicted_ranges = calculate_predicted_ranges(position, clock_error, satellite_positions)
    H = form_H_matrix(position, satellite_positions, predicted_ranges)
    calculated_pseudo_ranges = calculate_pseudo_ranges(position, clock_error, satellite_positions)
    return H @ (calculated_pseudo_ranges - measured_pseudo_ranges)

# Initial guess for the receiver's position (x, y, z) and clock error
initial_guess = np.array([0, 0, 0, 0])  # Clock error in meters, assuming the center of the Earth

# Perform least squares optimization
result = least_squares(residuals, initial_guess, args=(satellite_positions, measured_pseudo_ranges))

# The result contains the estimated receiver position and clock error
estimated_position = result.x[:3]
estimated_clock_error = result.x[3] / c  # Convert clock error back to seconds

# Constants for the WGS-84 ellipsoid
a = 6378137.0  # Semi-major axis
f = 1 / 298.257223563  # Flattening
e_squared = f * (2 - f)  # Eccentricity squared

def cartesian_to_curvilinear_corrected(x, y, z):
    # Calculations
    p = np.sqrt(x**2 + y**2)  # Distance to z-axis

    # Initial guess for latitude based on spherical earth
    lat0 = np.arctan2(z, p)

    # Iterative calculation of geodetic latitude
    N = a
    h = 0
    lat = lat0
    for _ in range(5):  # Five iterations should be enough for convergence
        N = a / np.sqrt(1 - e_squared * np.sin(lat)**2)
        h = p / np.cos(lat) - N
        lat = np.arctan2(z, p * (1 - e_squared * (N / (N + h))))

    # Longitude calculation
    lon = np.arctan2(y, x)

    # Convert radians to degrees for latitude and longitude
    lat_deg = np.degrees(lat)
    lon_deg = np.degrees(lon)

    return lat_deg, lon_deg, h

# Use the estimated ECEF position to calculate the latitude, longitude, and corrected height
latitude_deg, longitude_deg, height_meters = cartesian_to_curvilinear_corrected(*estimated_position)

# Outputs
print("Estimated Position (ECEF):", estimated_position)
print("Estimated Clock Error (seconds):", estimated_clock_error)
print("Latitude (degrees):", latitude_deg)
print("Longitude (degrees):", longitude_deg)
print("Height (meters):", height_meters)


Estimated Position (ECEF): [3816733.93367205  -65828.71542141 5092638.70460178]
Estimated Clock Error (seconds): -2.4162344607219496e-14
Latitude (degrees): 53.33030509840839
Longitude (degrees): -0.9881049356481474
Height (meters): 71.75928454659879
